# Building a recommendation engine using matrix factorization

Let us define a function to predict the ratings given by the user to all the movies which are not rated by him/her.

In [11]:
import numpy as np
import pandas as pd

In [12]:
# reading rating file
r_cols = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv('data/u.data',sep ='\t',names = r_cols)

In [35]:
class MF():
    #Initializing the user-movie rating matrix, no. of latent frature, alpha  and beta
    def __init__(self, R,K, alpha , beta , iterations):
        self.R = R
        self.num_users , self.num_items = R.shape
        self.K =K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
    #Intializing user-feature and movie-feature matrix
    
    #Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)
    
    #Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            
    # Rating for user i and movie j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    # FULL user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[: ,np.newaxis] + mf.b_i[np.newaxis: ,] +mf.P.dot(mf.Q.T)
    
    def train(self):
        self.P = np.random.normal(scale = 1./self.K , size=(self.num_users , self.K))
        self.Q = np.random.normal(scale = 1./self.K , size=(self.num_items , self.K))
        
        # initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i =np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        # List of training samples
        self.samples = [
            (i,j,self.R[i,j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i,j] > 0
        ]
        
        # Stochastic gradient decent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i,mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" %(i+1,mse))
        return training_process

        

Now we have a function that can predict the ratings. The input for this function are:

- R – The user-movie rating matrix
- K – Number of latent features
- alpha – Learning rate for stochastic gradient descent
- beta – Regularization parameter for bias
- iterations – Number of iterations to perform stochastic gradient descent

In [36]:
R= np.array(ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0))

In [ ]:
mf = MF(R, K=20, alpha=0.001, beta=0.01, iterations=1000)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 296.1126
Iteration: 40 ; error = 291.0028
Iteration: 60 ; error = 287.4403
Iteration: 80 ; error = 281.6394
Iteration: 100 ; error = 272.2601
Iteration: 120 ; error = 261.5457
Iteration: 140 ; error = 250.6097
Iteration: 160 ; error = 240.2238
Iteration: 180 ; error = 231.1355
Iteration: 200 ; error = 223.5613
Iteration: 220 ; error = 217.3400
Iteration: 240 ; error = 212.2039
Iteration: 260 ; error = 207.9129
Iteration: 280 ; error = 204.2836
Iteration: 300 ; error = 201.1806
Iteration: 320 ; error = 198.5034
Iteration: 340 ; error = 196.1751
Iteration: 360 ; error = 194.1360
Iteration: 380 ; error = 192.3396
Iteration: 400 ; error = 190.7454
